# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all


You are using Wmfdata v2.0.0, but v2.0.1 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md.


# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/readers_diversity_metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



In [3]:
datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

datetime.date(2023, 8, 31)

# Preparation

In [4]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [5]:
# Code to Suppress PySpark Warning messages
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
queries = {
    
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    },
    "global_north_previews": {
        "file": "queries/global_north_previews.hql",
        "engine": "hive"
    },
     "global_north_pageviews": {
        "file": "queries/global_north_pageviews.hql",
        "engine": "hive"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running global_south_pageviews on hive...



SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/21 23:04:18 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/09/21 23:04:18 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/09/21 23:04:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/21 23:04:27 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/09/21 23:04:27 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!
23/09/21 23:04:31 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
Running global_south_preview

Running global_north_previews on hive...                                        


Running global_north_pageviews on hive...                                       


# Combining and saving metrics

In [7]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)
new_metrics['gn_interactions'] = new_metrics.apply(lambda x: x['gn_previews'] + x['gn_pageviews'], axis=1)


In [8]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,gn_interactions,gn_pageviews,gn_previews,gs_interactions,gs_pageviews,gs_previews
month,,,,,,
2022-11-01,1.527185e+10,1.386750e+10,1.404349e+09,5.738306e+09,5.417234e+09,321072015.0
2022-12-01,1.419016e+10,1.285189e+10,1.338267e+09,4.324661e+09,4.018421e+09,306240248.0
2023-01-01,1.571824e+10,1.418498e+10,1.533265e+09,4.789970e+09,4.470099e+09,319871193.0
2023-02-01,1.397315e+10,1.262389e+10,1.349264e+09,4.968715e+09,4.675047e+09,293667680.0
2023-03-01,1.553058e+10,1.404855e+10,1.482034e+09,5.560632e+09,5.234402e+09,326229545.0
2023-04-01,1.396267e+10,1.262565e+10,1.337021e+09,4.601339e+09,4.303320e+09,298019127.0
2023-05-01,1.508696e+10,1.367465e+10,1.412308e+09,5.760060e+09,5.434040e+09,326020485.0
2023-06-01,1.425471e+10,1.299977e+10,1.254937e+09,5.297903e+09,5.004453e+09,293449220.0
2023-07-01,1.469120e+10,1.344263e+10,1.248574e+09,5.400382e+09,5.101558e+09,298824401.0


In [9]:
metrics.to_csv(FILENAME, sep="\t")